In [3]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# ===============================
# 1️⃣ 데이터 경로 및 라벨별 리스트
# ===============================
data_dir = "/mnt/nas/seohan_image/seohan_data/origin/"
labels = ['27_45','27_30']

image_paths = []
image_labels = []

for label in labels:
    label_dir = os.path.join(data_dir, label)
    for img_name in os.listdir(label_dir):
            image_paths.append(os.path.join(label_dir, img_name))
            image_labels.append(label)

# ===============================
# 2️⃣ train/val 나누기 (80:20)
# ===============================
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, image_labels, test_size=0.2, stratify=image_labels, random_state=42
)


/home/dlit/miniconda3/envs/pgn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
train_transform = A.Compose([
    A.ShiftScaleRotate(
        shift_limit=0.08, scale_limit=0.05, rotate_limit=10, border_mode=cv2.BORDER_REFLECT_101, p=0.8
    ),
    A.MotionBlur(blur_limit=5, p=0.3),
    A.GaussianBlur(blur_limit=3, p=0.2),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.3),  # 색온도 변화 대응
    A.CLAHE(p=0.2),  # 조명 보정
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.Resize(299, 299),
    A.Normalize(),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(299, 299),
    A.Normalize(),
    ToTensorV2()
])


/home/dlit/miniconda3/envs/pgn/lib/python3.9/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [5]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        self.label_to_idx = {l: i for i, l in enumerate(sorted(set(labels)))}
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image = np.array(Image.open(self.image_paths[idx]).convert('RGB'))
        label = self.label_to_idx[self.labels[idx]]
        if self.transform:
            image = self.transform(image=image)['image']
        return image, label

train_dataset = CustomDataset(train_paths, train_labels, transform=train_transform)
val_dataset = CustomDataset(val_paths, val_labels, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)


In [6]:
import torch
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.inception_resnet_v2(weights='IMAGENET1K_V1')  # 최신 torchvision 기준
model.fc = nn.Linear(model.fc.in_features, 2)  # 라벨 2개

# 초기엔 feature extractor 동결
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

model = model.to(device)


AttributeError: module 'torchvision.models' has no attribute 'inception_resnet_v2'

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch
import torch.nn as nn

# ===============================
# (1) 라벨 데이터로 class weight 계산
# ===============================
# train_labels: 문자열 리스트 예) ["label1", "label2", "label2", ...]
label_to_idx = {'label1': 0, 'label2': 1}
y_train = np.array([label_to_idx[l] for l in train_labels])

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
print(f"Class Weights: {class_weights}")

# ===============================
# (2) 손실 함수 정의 시 weight 적용
# ===============================
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)


In [ ]:
def tta_predict(model, image, n=5):
    model.eval()
    preds = []
    for _ in range(n):
        aug_img = train_transform(image=image)['image'].unsqueeze(0).to(device)
        with torch.no_grad():
            preds.append(torch.softmax(model(aug_img), dim=1).cpu().numpy())
    return np.mean(preds, axis=0)


In [21]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
import cv2
from tqdm import tqdm

# ===============================
# ⚙️ 기본 설정
# ===============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = 32
num_epochs = 20
num_workers = 4

# ===============================
# 🧩 1. 데이터 경로 수집
# ===============================
data_dir = "/mnt/nas/seohan_image/seohan_data/origin/"
labels = ['27_45','27_30']
save_dir = './new_model'

os.makedirs(save_dir, exist_ok=True)

image_paths, image_labels = [], []

for label in labels:
    label_dir = os.path.join(data_dir, label)
    for img_name in os.listdir(label_dir):
            image_paths.append(os.path.join(label_dir, img_name))
            image_labels.append(label)

# 라벨 인덱싱
label_to_idx = {l: i for i, l in enumerate(sorted(set(image_labels)))}
y_all = np.array([label_to_idx[l] for l in image_labels])

# ===============================
# 🧩 2. Stratified Split (80:20)
# ===============================
train_paths, val_paths, y_train, y_val = train_test_split(
    image_paths, y_all, test_size=0.2, stratify=y_all, random_state=42
)

# ===============================
# 🧠 3. Class Weight 계산
# ===============================
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
print(f"✅ Class Weights: {class_weights}")

# ===============================
# 🎥 4. Albumentations 증강 설정
# ===============================
train_transform = A.Compose([
    A.ShiftScaleRotate(shift_limit=0.08, scale_limit=0.05, rotate_limit=10, border_mode=cv2.BORDER_REFLECT_101, p=0.8),
    A.MotionBlur(blur_limit=5, p=0.3),
    A.GaussianBlur(blur_limit=3, p=0.2),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.CLAHE(p=0.2),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.CenterCrop(height=1900, width=1900),
    A.Resize(299, 299),
    ToTensorV2()
])

val_transform = A.Compose([
    A.CenterCrop(height=1900, width=1900),
    A.Resize(299, 299),
    ToTensorV2()
])

# ===============================
# 📦 5. Dataset 정의
# ===============================
class CustomDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, idx):
        image = np.array(Image.open(self.paths[idx]).convert("RGB"), dtype=np.float32) / 255.0
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image=image)['image']
        return image, label

train_dataset = CustomDataset(train_paths, y_train, transform=train_transform)
val_dataset = CustomDataset(val_paths, y_val, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# ===============================
# 🧱 6. timm 모델 불러오기
# ===============================
model = timm.create_model("inception_resnet_v2", pretrained=True, num_classes=len(label_to_idx))
model = model.to(device)

# ===============================
# ⚙️ 7. 손실함수 & 옵티마이저
# ===============================
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# ===============================
# 🚀 8. 학습 루프
# ===============================
best_val_acc = 0.0
for epoch in range(num_epochs):
    model.train()
    train_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc=f"[Epoch {epoch+1}] Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_acc = 100 * correct / total
    avg_train_loss = train_loss / total

    # -----------------------
    # Validation
    # -----------------------
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    
    val_acc = 100 * val_correct / val_total
    avg_val_loss = val_loss / val_total

    print(f"📊 Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {avg_train_loss:.4f}, Acc: {train_acc:.2f}% | "
          f"Val Loss: {avg_val_loss:.4f}, Acc: {val_acc:.2f}%")


    # ---- 스케줄러 갱신 ----
    scheduler.step()

    # ---- 모델 저장 ----
    save_path = os.path.join(save_dir, f"epoch_{epoch:03d}_val_{val_acc:.2f}.pt")
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'val_acc': val_acc,
        'optimizer_state_dict': optimizer.state_dict(),
    }, save_path)
    print(f"💾 모델 저장 완료: {save_path}")

    # ---- 최고 성능 모델 별도 저장 ----
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), os.path.join(save_dir, "best_model.pt"))
        print(f"🌟 최고 성능 갱신! Best Acc: {best_val_acc:.2f}%")

torch.save(model.state_dict(), os.path.join(save_dir, "final_model.pt"))
print("✅ 학습 완료")


✅ Class Weights: tensor([0.6983, 1.7610], device='cuda:0')


/home/dlit/miniconda3/envs/pgn/lib/python3.9/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
[Epoch 1] Training: 100%|██████████| 1168/1168 [39:23<00:00,  2.02s/it]


📊 Epoch [1/20] Train Loss: 0.0287, Acc: 98.97% | Val Loss: 0.0003, Acc: 100.00%
💾 모델 저장 완료: ./new_model/epoch_000_val_100.00.pt
🌟 최고 성능 갱신! Best Acc: 100.00%


[Epoch 2] Training: 100%|██████████| 1168/1168 [38:39<00:00,  1.99s/it]


📊 Epoch [2/20] Train Loss: 0.0047, Acc: 99.86% | Val Loss: 0.0036, Acc: 99.99%
💾 모델 저장 완료: ./new_model/epoch_001_val_99.99.pt


...

📊 Epoch [18/20] Train Loss: 0.0000, Acc: 100.00% | Val Loss: 0.0001, Acc: 100.00%
💾 모델 저장 완료: ./new_model/epoch_017_val_100.00.pt


[Epoch 19] Training: 100%|██████████| 1168/1168 [37:45<00:00,  1.94s/it]


📊 Epoch [19/20] Train Loss: 0.0000, Acc: 100.00% | Val Loss: 0.0000, Acc: 100.00%
💾 모델 저장 완료: ./new_model/epoch_018_val_100.00.pt


[Epoch 20] Training: 100%|██████████| 1168/1168 [37:41<00:00,  1.94s/it]


📊 Epoch [20/20] Train Loss: 0.0000, Acc: 100.00% | Val Loss: 0.0000, Acc: 100.00%
💾 모델 저장 완료: ./new_model/epoch_019_val_100.00.pt
✅ 학습 완료
